<div class="alert alert-block alert-success">
<h3> Exercise 1: Solve the balls and bin problem via an integer program</h3>
<p>Assume that you have the data for balls and bins provided to you.</p>
<p>Solve to find the best assignment of balls into bins</p>
</div>

<img src="imgs/Balls.png" width="60%">

<img src="imgs/Bins.png" width="60%">

### Rules
<ol>
<li>Putting a ball into a bin gives you a reward equal to their product (ball score X bin score).
<li>Balls can go into bins of matching colors only.
<li>Small bin can accept only one small ball.
<li> Big bin can accept either one large ball or two small balls.
</ol>

In [63]:
from gurobipy import *

def solve(elements, positions, frequency, distance):
    # ==== 1. Create the (empty) model ====
    model = Model("linear_menu")

    # ==== 2. Add decision variables ======
    x = {}
    # Create one binary variable for each element-position pair.
    # We give it a meaningful name so we later understand what it means
    # if it is set to 1
    for e in elements:
        for p in positions:
            x[(e, p)] = model.addVar(vtype=GRB.BINARY, name="%s_%s" % (e, p))
            # Integrate new variables
    model.update()

    # ====3. Add Constraints ======
    # Add constraints
    
    # 1. Putting a ball into a bin gives you a reward equal to their product (ball score X bin score).
    # 2. Balls can go into bins of matching colors only.
    # 3. Small bin can accept only one small ball.
    # 4. Big bin can accept either one large ball or two small balls.
    
    # Ball colors: 0:red, 1:yellow, 2:yellow, 3:yellow(big), 4:yellow 
    # Bin colors: 0:red, 1:yellow, 2:both
    
    # bin A can only accept ball 1
    model.addConstr(x[("2","A")] == 0, "A cannot accept 2-5")
    model.addConstr(x[("3","A")] == 0, "A cannot accept 2-5")
    model.addConstr(x[("4","A")] == 0, "A cannot accept 2-5")
    model.addConstr(x[("5","A")] == 0, "A cannot accept 2-5")
    # bin B cannot accept ball 1
    model.addConstr(x[("1","B")] == 0, "B cannot accept 1")
    # bin B cannot accept ball 4
    model.addConstr(x[("4","B")] == 0, "B cannot accept 4")
    # bin B can only hold one ball
    #model.addConstr(quicksum(x[("B", e)] for e in elements) <= 1, "B can accept only one ball")
    model.addConstr(quicksum(x[(e,"B")] for e in elements) <= 1, "B can accept only one ball")
    # bin A can hold two small balls (though only 1 red ball exists)
    #
    # bin C can hold two balls, or ball 4
    model.addConstr((quicksum(x[(e, "C")] for e in ['1', '2', '3', '5']) + 2 * x[("4", "C")]) <= 2, "C can accept two balls, or ball 4")
    
    # Each element is only assigned to one position
    for p in positions: 
        model.addConstr(quicksum(x[(e,p)]
                   for e in elements) == 1, "Balls cannot be in two places")  
    
    # Integrate new variables
    model.update()
    
    # ==== 4. Specify Objective function ======
    # Maximise ball placement
    target = quicksum(frequency[e] * distance[p]  * x[(e, p)]
                    for e in elements
                    for p in positions)
    model.setObjective(target, GRB.MAXIMIZE)

    
    # ==== 5. Optimize model ======    
    model.optimize()
    
    # ====6. Extract solution ======
    layout = [None] * len(elements)
    # create the layout (ordered list of elements) from the variables
    # that are set to 1
    layout = []
    for v in model.getVars():
        if v.x == 1:
            element = v.varName.split("_")[0]
            position = v.varName.split("_")[1]
            layout.append([element, position])

    return layout, model.getObjective().getValue()

#elements
BALLS = ["1", "2", "3", "4", "5"]
#positions
BINS = ["A", "B", "C"]

#cost factors
#frequency
BALL_POINTS = {"1":1, "2":3, "3":3, "4":5, "5":3}
#distance
BIN_POINTS = {"A":5, "B":10, "C":1}

#solve the problem
result, objective = solve(BALLS, BINS, BALL_POINTS, BIN_POINTS)

# solve the problem
result, objective = solve(elements, positions, frequency, distance)
#Print the solution
print "Objective value (expected selection time):", objective
# Print result
print "Result is: ", result

Optimize a model with 11 rows, 15 columns and 31 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 38
Presolve removed 11 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.09 seconds
Thread count was 1 (of 1 available processors)

Solution count 2: 40 38 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+01, best bound 4.000000000000e+01, gap 0.0000%
Optimize a model with 11 rows, 15 columns and 31 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 38
Presolve removed 11 rows an

<div class="alert alert-block alert-success">
<h3> Exercise 2: Trading off different users</h3>
<p>Assume that you have two different user groups that use the menu in very different ways, e.g. novice versus expert users. </p>
<p>Given two sets of frequency distributions for the menu items, $p^{novice}$ and $p^{expert}$ your task is to reformulate the objective function so that it finds the best design for both user groups. </p>
</div>


<div class="alert alert-block alert-success">
<h3> Exercise 3a: Modeling the letter assignment problem</h3>
How can you model the problem of assigning characters to keyslots on a keyboard mathematically?
<ol>
<li> Define the decision variables
<li> Add the necessary constraints
<li> Formulate the objective function. 
</ol>
</div>

<br>
<div class="alert alert-block alert-success">
<h3> Exercise 3b: Implementing the letter assignment problem in Gurobi</h3>
Implement the model in Gurobi and optimize a keyboard layout for the given letters.
</div>

<div class="alert alert-block alert-success">
<h3> Bonus task:</h3>
<p>We really want to name our keyboard the "HCI" keyboard.</p>
<br>
<p>Therefore, your task is to change the mathematical model and its implementation so that the letters H - C - I are placed next to each other on any of the rows of the computer, as in the example keyboard below. Do not change the input data.</p>
<p>
How much worse is this keyboard in comparison to the unconstrained problem?</p>                                                 
</div>